### Last digit of the name in label data is the target label 

In [ ]:
import os

In [ ]:
nums = os.listdir('data')
nums = [os.path.splitext(s)[0][-1] for s in nums]
import pandas as pd
nums = pd.DataFrame(nums)
print(len(os.listdir('data')))
nums.value_counts()

In [ ]:
# import cv2
# import skimage
# import matplotlib.pyplot as plt
# from utils import *
# digits = []
# labels = []
# rel_dir = 'raw label'
# for filename in os.listdir(rel_dir)[:]:
#     img = cv2.imread(os.path.join(rel_dir, filename))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#     img = cv2.resize(img, (100,100))
#     img = unpad(img, 15)
#     img = cv2.resize(img, (28,28))
#     digits.append(img)
#     labels.append(os.path.splitext(filename)[0][-1])
# save_label_digit(digits, labels)
# display_digits_line(digits,labels, '', len(labels))

In [ ]:
from utils import *
import cv2
rel_dir = 'raw image'
filename = 'sca3.jpg'

img = cv2.imread(f'{rel_dir}/{filename}')
digits = crop_number(img.copy(), True)

In [ ]:
labelled_digits, labels = manual_label(digits[50:75])

In [ ]:
final_digits = transform_cropped_digit(labelled_digits, labels)
display_digits_line(final_digits, labels, '', len(labels))

In [ ]:
save_label_digit(final_digits, labels)